In [206]:
import numpy as np
import pandas as pd
import datetime
import operator

## instance test

In [297]:
grid = pd.DataFrame(index=range(9), columns=range(9), data=np.NaN)
square = grid.copy()
square = square.unstack().reset_index()
square[0] = (square['level_0'] // 3) + (square['level_1'] // 3) * 3
square.rename(columns={'level_0': 'index', 'level_1': 'columns', 0:'group'}, inplace=True)
square['value'] = np.NaN
# what we created, though it is more exploitable in non pivoted view
square.pivot_table(index='index', columns='columns', values='group')

columns,0,1,2,3,4,5,6,7,8
index,,,,,,,,,
0,0,0,0,3,3,3,6,6,6
1,0,0,0,3,3,3,6,6,6
2,0,0,0,3,3,3,6,6,6
3,1,1,1,4,4,4,7,7,7
4,1,1,1,4,4,4,7,7,7
5,1,1,1,4,4,4,7,7,7
6,2,2,2,5,5,5,8,8,8
7,2,2,2,5,5,5,8,8,8
8,2,2,2,5,5,5,8,8,8


In [179]:
grid.loc[0,0] = 5
grid.loc[0,1] = 3
grid.loc[0,4] = 7

grid.loc[1,0] = 6
grid.loc[1,3] = 1
grid.loc[1,4] = 9
grid.loc[1,5] = 5

grid.loc[2,1] = 9
grid.loc[2,2] = 8
grid.loc[2,7] = 6

grid.loc[3,0] = 8
grid.loc[3,4] = 6
grid.loc[3,8] = 3

grid.loc[4,0] = 4
grid.loc[4,3] = 8
grid.loc[4,5] = 3
grid.loc[4,8] = 1

grid.loc[5,0] = 7
grid.loc[5,4] = 2
grid.loc[5,8] = 6


grid.loc[6,1] = 6
grid.loc[6,6] = 2
grid.loc[6,7] = 8

grid.loc[7,3] = 4
grid.loc[7,4] = 1
grid.loc[7,5] = 9
grid.loc[7,8] = 5

grid.loc[8,4] = 8
grid.loc[8,7] = 7
grid.loc[8,8] = 9


In [198]:
grid

,0,1,2,3,4,5,6,7,8
0,5.0,3.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN
1,6.0,NaN,NaN,1.0,9.0,5.0,NaN,NaN,NaN
2,NaN,9.0,8.0,NaN,NaN,NaN,NaN,6.0,NaN
3,8.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,3.0
4,4.0,NaN,NaN,8.0,NaN,3.0,NaN,NaN,1.0
5,7.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.0
6,NaN,6.0,NaN,NaN,NaN,NaN,2.0,8.0,NaN
7,NaN,NaN,NaN,4.0,1.0,9.0,NaN,NaN,5.0
8,NaN,NaN,NaN,NaN,8.0,NaN,NaN,7.0,9.0


## utils

In [ ]:
def first_nan(grid):
    """
    find a NaN position in the grid
    safe policy
    """
    row = grid.isnull().sum(axis=1).argmax()
    col = grid.loc[row, :].isnull().argmax()
    return row, col

## First try:
### Naive approach

the first try is a very naive one

In [169]:
def is_feasible(grid):
    # 1 by row
    if (grid.nunique(axis=0) != grid.count(axis=0)).any():
        return False
    # 1 by column
    if (grid.nunique(axis=1) != grid.count(axis=1)).any():
        return False
    # 1 by square
    tmp = grid.unstack().reset_index()
    tmp['group'] = square['group']
    if (tmp.groupby(['group'])[0].value_counts().max(level=0) > 1).any():
        return False

    return True

def solve(grid):
    # end this path is infeasible
    if not is_feasible(grid):
        return False, None

    # if the grid is complete we are done :)
    if grid.isnull().sum().sum() == 0:
        return True, grid

    else:
        # find a NaN position
        idx, col = first_nan(grid)
        new_grid = grid.copy()
        # try every value on this position
        for i in range(1,10):
            new_grid.loc[idx, col] = i
            feasible, sol = solve(new_grid)
            if feasible:
                break

    return feasible, sol

In [300]:
start = datetime.datetime.now()
print(solve(grid))
print(datetime.datetime.now() - start)

(True,      0    1    2    3    4    5    6    7    8
0  1.0  4.0  7.0  2.0  3.0  8.0  5.0  6.0  9.0
1  2.0  5.0  8.0  1.0  6.0  9.0  3.0  4.0  7.0
2  3.0  6.0  9.0  4.0  5.0  7.0  1.0  2.0  8.0
3  4.0  7.0  1.0  3.0  8.0  2.0  6.0  9.0  5.0
4  5.0  8.0  2.0  6.0  9.0  1.0  4.0  7.0  3.0
5  6.0  9.0  3.0  5.0  7.0  4.0  2.0  8.0  1.0
6  7.0  1.0  4.0  8.0  2.0  3.0  9.0  5.0  6.0
7  8.0  2.0  5.0  9.0  1.0  6.0  7.0  3.0  4.0
8  9.0  3.0  6.0  7.0  4.0  5.0  8.0  1.0  2.0)
0:00:13.898746


## Second try: 
### we filter possibilities rather than check the feasibility

we only try acceptable values instead of trying all of them then checking

In [255]:
def solve2(grid):
    # end this path is infeasible
    if not is_feasible(grid):
        return False, None

    # if the grid is complete we are done :)
    if grid.isnull().sum().sum() == 0:
        return True, grid

    else:
        # find a NaN position
        idx, col = first_nan(grid)
        new_grid = grid.copy()
        # test only possible values
        possible_values = set(range(1,10))
        possible_values -= set(new_grid.loc[idx, :].to_list())
        possible_values -= set(new_grid.loc[:, col].to_list())
        possible_values -= set(new_grid.loc[(idx//3)*3:(idx//3)*3+2, (col//3)*3:(col//3)*3+2].values.flatten())
        # is no value possible: the path is infeasible
        if len(possible_values) == 0:
            return False, None
        else:
            for i in possible_values:
                new_grid.loc[idx, col] = i
                feasible, sol = solve2(new_grid)
                if feasible:
                    break
    return feasible, sol

In [299]:
start = datetime.datetime.now()
print(solve2(grid))
print(datetime.datetime.now() - start)

(True,      0    1    2    3    4    5    6    7    8
0  1.0  4.0  7.0  2.0  3.0  8.0  5.0  6.0  9.0
1  2.0  5.0  8.0  1.0  6.0  9.0  3.0  4.0  7.0
2  3.0  6.0  9.0  4.0  5.0  7.0  1.0  2.0  8.0
3  4.0  7.0  1.0  3.0  8.0  2.0  6.0  9.0  5.0
4  5.0  8.0  2.0  6.0  9.0  1.0  4.0  7.0  3.0
5  6.0  9.0  3.0  5.0  7.0  4.0  2.0  8.0  1.0
6  7.0  1.0  4.0  8.0  2.0  3.0  9.0  5.0  6.0
7  8.0  2.0  5.0  9.0  1.0  6.0  7.0  3.0  4.0
8  9.0  3.0  6.0  7.0  4.0  5.0  8.0  1.0  2.0)
0:00:04.474437


better, still disappointing

## Third try:
### we optimize order of visited cells

let chose more cleverly the next cell to explore

In [295]:
def compute_possible_values(grid, idx, col):
    possible_values = set(range(1,10))
    possible_values -= set(grid.loc[idx, :].to_list())
    possible_values -= set(grid.loc[:, col].to_list())
    possible_values -= set(grid.loc[(idx//3)*3:(idx//3)*3+2, (col//3)*3:(col//3)*3+2].values.flatten())
    return possible_values

def first_nan2(grid):
    """
    find all available numbers for each position still unaffected.
    If we find a position with only one possibility we stop and go with this position/number
    """
    res = {}
    # test all position with NAN
    # look a bit better than grid.index: sort index by nan value: more chances to 
    # find a fully determined cell in a row with a lot of values
    for i in grid.isnull().sum(axis=1).sort_values().index:
        # doesnt look worth: grid.isnull().sum(axis=0).sort_values().index:
        for j in grid.columns: 
            if pd.isnull(grid.loc[i, j]):
                candidates = compute_possible_values(grid, i, j)
                # we found an optimal position
                if len(candidates) ==1:
                    return i, j, candidates
                else:
                    res[(i,j)] = candidates
    # return pos with the least candidates
    (row, col), candidates = min(res.items(), key=operator.itemgetter(1))
    return row, col, candidates

def solve3(grid):
    # end this path is infeasible
    if not is_feasible(grid):
        return False, None

    # if the grid is complete we are done :)
    if grid.isnull().sum().sum() == 0:
        return True, grid

    else:
        # find best candidate
        idx, col, candidates = first_nan2(grid)
        new_grid = grid.copy()
        if len(candidates) == 0:
            return False, None
        else:
            for i in candidates:
                new_grid.loc[idx, col] = i
                feasible, sol = solve3(new_grid)
                if feasible:
                    break
    return feasible, sol

In [298]:
start = datetime.datetime.now()
print(solve3(grid))
print(datetime.datetime.now() - start)

(True,      0    1    2    3    4    5    6    7    8
0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0
1  4.0  5.0  6.0  7.0  8.0  9.0  1.0  2.0  3.0
2  7.0  8.0  9.0  1.0  2.0  3.0  4.0  5.0  6.0
3  2.0  1.0  4.0  3.0  6.0  5.0  8.0  9.0  7.0
4  3.0  6.0  5.0  8.0  9.0  7.0  2.0  1.0  4.0
5  8.0  9.0  7.0  2.0  1.0  4.0  3.0  6.0  5.0
6  5.0  3.0  1.0  6.0  4.0  2.0  9.0  7.0  8.0
7  6.0  4.0  2.0  9.0  7.0  8.0  5.0  3.0  1.0
8  9.0  7.0  8.0  5.0  3.0  1.0  6.0  4.0  2.0)
0:00:02.227223


looks more like what i was initially expecting :)